# Heat Exchanger 1D Unit Model with Two Property Packages

![](heat_exchanger_4.svg)

**Problem Statement**: In this example, we will be heating a benzene-toluene mixture using steam. 

**Tube Side Inlet**

Flow Rate = 250 mol/s

Mole fraction (Benzene) = 0.4

Mole fraction (Toluene) = 0.6

Pressure = 101325 Pa

Temperature = 350 K

**Shell Side Inlet**

Flow Rate = 100 mol/s

Mole fraction (Steam) = 1

Pressure = 101325 Pa

Temperature = 450 K

This example will demonstrate the simulation of the 1D heat exchanger by fixing any 7 of the following degrees of freedom:
- two of shell length, diameter, and area
- two of tube length, diameter, and area
- number of tubes
- wall temperature (at all spatial points)
- heat transfer coefficient (at all spatial points, for both shell and tube)


IDAES documentation reference for heat exchanger 1D model: https://idaes-pse.readthedocs.io/en/latest/reference_guides/model_libraries/generic/unit_models/heat_exchanger_1D.html

This example utilizes the simple 1D heat exchanger for a shell and tube system. The IDAES library contains a more advanced `ShellAndTube1D` heat exchanger supporting a 0D wall conduction model; more details on the advanced 1D heat exchanger may be found [here](https://idaes-pse.readthedocs.io/en/latest/reference_guides/model_libraries/generic/unit_models/shell_and_tube_1d.html).

## Setting up the problem in IDAES

First, import the required IDAES and Pyomo modules. Note that the hotside (shell) and coldside (tube) properties leverage separate property packages:

In [ ]:
# Import pyomo package 
from pyomo.environ import ConcreteModel, Constraint, value, units

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import the IAPWS property package to create a properties block for the flowsheet
from idaes.models.properties import iapws95

from idaes.models.properties.iapws95 import htpx

from idaes.models.properties.modular_properties.base.generic_property import (
        GenericParameterBlock)

from idaes.models.properties.modular_properties.examples.BT_ideal \
    import configuration

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
from idaes.core.util.model_statistics import degrees_of_freedom

# Import the default IPOPT solver
from idaes.core.solvers import get_solver

# Import a heat exchanger unit
from idaes.models.unit_models.heat_exchanger_1D import (HeatExchanger1D,
                                                        HeatExchangerFlowPattern)

Then, build the model and flowsheet components using the imported property packages:

In [ ]:
# Create the ConcreteModel and the FlowsheetBlock, and attach the flowsheet block to it.
m = ConcreteModel()

# Steady State Model
m.fs = FlowsheetBlock(dynamic=False)

# Setup property packages for shell and tube side
# Steam property package
m.fs.properties_shell = iapws95.Iapws95ParameterBlock()

# BT ideal property package
m.fs.properties_tube = GenericParameterBlock(**configuration)

In the [0D Heat Exchanger model example](http://localhost:8888/notebooks/GitHub/examples-pse/src/Examples/UnitModels/Operations/heat_exchanger_0D_testing.ipynb), geometry effects are ignored in favor of temperature gradients to calculate heat transfer. Here, we need to specify discretization, a flow configuration and wall conduction assumption. We specify the one-dimensional spatial discretization to use backwards finite difference approximations with 20 finite elements - these are the defaults if none are specified explicitly, and the shell and tube domains may be discretized differently if desired. The domains must use the same number of finite elements, as the exchanger is linear and the elements directly correspond according to the selected flow pattern.

The 1D Heat Exchanger supports the following flow configuration options:

- HeatExchangerFlowPattern.cocurrent. Shell and tube flow in parallel, tube inlet transfers with shell inlet and similar with outlets, and temperature difference is greatest at the flow inlets (default).
- HeatExchangerFlowPattern.countercurrent. Shell and tube flow in anti-parallel, tube inlet transfers with shell outlet and vice versa, and temperature difference changes minimally along the exchanger length.

The unit is created below:

In [ ]:
# Create an instance of the heat exchanger unit, attaching it to the flowsheet
# Specify that the property packages to be used with the heat exchanger are the ones we created earlier.
m.fs.heat_exchanger = HeatExchanger1D(
    hot_side_name="shell",
    cold_side_name="tube",
    shell={"property_package": m.fs.properties_shell,
           "transformation_method": "dae.finite_difference",
           "transformation_scheme": "BACKWARD"},
    tube={"property_package": m.fs.properties_tube,
          "transformation_method": "dae.finite_difference",
          "transformation_scheme": "BACKWARD"},
    finite_elements=20,
    flow_type=HeatExchangerFlowPattern.cocurrent)

# Call the degrees_of_freedom function, get initial DOF
DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

In [ ]:
assert DOF_initial == 31

## Fixing input specifications
For this problem, we will fix the inlet conditions and run two different options for unit specifications:

In [ ]:
h = htpx(450*units.K, P = 101325*units.Pa)  # calculate IAPWS enthalpy

# Fix the inlet conditions
m.fs.heat_exchanger.hot_side_inlet.flow_mol.fix(100) # mol/s
m.fs.heat_exchanger.hot_side_inlet.pressure.fix(101325) # Pa
m.fs.heat_exchanger.hot_side_inlet.enth_mol.fix(h) # J/mol

DOF_initial = degrees_of_freedom(m)
print("The DOF is {0}".format(DOF_initial))

In [ ]:
m.fs.heat_exchanger.cold_side_inlet.flow_mol.fix(250) # mol/s
m.fs.heat_exchanger.cold_side_inlet.mole_frac_comp[0, "benzene"].fix(0.4)
m.fs.heat_exchanger.cold_side_inlet.mole_frac_comp[0, "toluene"].fix(0.6)
m.fs.heat_exchanger.cold_side_inlet.pressure.fix(101325) # Pa
m.fs.heat_exchanger.cold_side_inlet.temperature[0].fix(350) # K

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print("The DOF is {0}".format(DOF_final))

### Option 1: Fix heat transfer coefficient (HTC) and dimensions of each domain
Below, we fix the heat exchanger area, length and heat transfer coefficient, which yields a fully defined problem for all finite elements with zero degrees of freedom that may be initialized and solved:

In [ ]:
m.fs.heat_exchanger.area.fix(0.5) # m2
m.fs.heat_exchanger.length.fix(4.85) # m
m.fs.heat_exchanger.heat_transfer_coefficient.fix(500) # W/m2/K

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print("The DOF is {0}".format(DOF_final))

In [ ]:
assert DOF_final == 0

Now that the problem is square (zero degress of freedom), we can initialize and solve the full model:

In [ ]:
# Initialize the flowsheet, and set the output at INFO
m.fs.heat_exchanger.initialize(outlvl=idaeslog.INFO)

# Solve the simulation using ipopt
# Note: If the degrees of freedom = 0, we have a square problem
opt = get_solver()
solve_status = opt.solve(m, tee = True)

# Display a readable report
m.fs.heat_exchanger.report()

In [ ]:
from pyomo.environ import assert_optimal_termination
import pytest

# Check if termination condition is optimal
assert_optimal_termination(solve_status)

assert value(m.fs.heat_exchanger.hot_side_outlet.enth_mol[0]) == pytest.approx(
    htpx(444.47*units.K, P = 101325*units.Pa), rel=1e-3)
assert value(m.fs.heat_exchanger.cold_side_outlet.temperature[0]) == pytest.approx(
    368.39, rel=1e-3)

### Option 2: Unfix shell length  and fix shell outlet temperatures

In the previous case, we fixed the heat exchanger area, length and overall heat transfer coefficient. However, given that the models in IDAES are equation oriented, we can fix the outlet variables. For example, we can fix the hot outlet temperature and heat exchanger length, and solve for the heat exchanger area that will satisfy that condition.


In [ ]:
m.fs.heat_exchanger.area.unfix()
m.fs.heat_exchanger.hot_side_outlet.enth_mol.fix(htpx(444.47*units.K, P = 101325*units.Pa))

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print("The DOF is {0}".format(DOF_final))

In [ ]:
result = opt.solve(m)

print(result)

# Display a readable report
m.fs.heat_exchanger.report()

In [ ]:
# Check if termination condition is optimal
assert_optimal_termination(result)

assert value(m.fs.heat_exchanger.area) == pytest.approx(0.5, abs=1e-2)
assert value(m.fs.heat_exchanger.cold_side_outlet.temperature[0]) == pytest.approx(
    368.39, rel=1e-3)